In [1]:
#Python packages
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 400)
import re
from scipy import stats

#Visualization packages
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#ML modeling packages
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [2]:
train = pd.read_csv('train_to_encode.csv')

In [3]:
train = train.iloc[:,1:]
train.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,weekday,month,day_of_year,wind_chill,heat_index,feels,radiation,altitude,solar_spec,hourly_roc,random,age
0,0,Electricity,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,4,1,1,83.6,77.6,77.0,1414.0,-51.78,NaN,1.7840,24802456,8.0
1,1,Electricity,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,4,1,1,83.6,77.6,77.0,1414.0,-51.78,NaN,1.2330,61316517,12.0
2,2,Electricity,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,4,1,1,83.6,77.6,77.0,1414.0,-51.78,NaN,0.9410,63831873,25.0
3,3,Electricity,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,4,1,1,83.6,77.6,77.0,1414.0,-51.78,NaN,0.7153,56365151,14.0
4,4,Electricity,0.0,0,Education,-14465,1975.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,4,1,1,83.6,77.6,77.0,1414.0,-51.78,NaN,0.8364,4040317,41.0


In [4]:
trainf = pd.read_csv('weather_train.csv')

In [5]:
train['timestamp']= trainf.timestamp

This data is weired until May 20. It is reported in this discussion by @barnwellguy that All electricity meter is 0 until May 20 for site_id == 0. I will remove these data from training data.

It corresponds to building_id <= 104.

In [5]:
train = train.query('not (building_id <= 104 & meter == 0 & timestamp <= "2016-05-20")')

In [ ]:
'''Since we've already took advantage of the converted meteorologcial features, we can now drop them.
This is also applicable to the "timestamp" feature'''

In [6]:
train.drop(['timestamp', 'air_temperature_f','wind_speed_mph','dew_temp_f','precip_cm'], axis=1, inplace=True)

In [10]:
train.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,weekday,month,day_of_year,wind_chill,heat_index,feels,radiation,altitude,solar_spec,hourly_roc,random,age
0,0,Electricity,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,4,1,1,83.6,77.6,77.0,1414.0,-51.78,NaN,1.7840,24802456,8.0
1,1,Electricity,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,4,1,1,83.6,77.6,77.0,1414.0,-51.78,NaN,1.2330,61316517,12.0
2,2,Electricity,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,4,1,1,83.6,77.6,77.0,1414.0,-51.78,NaN,0.9410,63831873,25.0
3,3,Electricity,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,4,1,1,83.6,77.6,77.0,1414.0,-51.78,NaN,0.7153,56365151,14.0
4,4,Electricity,0.0,0,Education,-14465,1975.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,4,1,1,83.6,77.6,77.0,1414.0,-51.78,NaN,0.8364,4040317,41.0


In [11]:
train.groupby('site_id').apply(lambda group: group.isna().sum())

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,weekday,month,day_of_year,wind_chill,heat_index,feels,radiation,altitude,solar_spec,hourly_roc,random,age
site_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,1076663,309,467404,309,104,10295,31250,0,0,0,0,0,0,0,0,0,0,10295,10399,1076559,0,0
1,0,0,0,0,0,0,140530,0,1386,446202,1386,553357,4599,1512,1323,0,0,0,0,0,1323,0,0,0,4599,553357,553294,0,140530
2,0,0,0,0,0,0,581932,2530312,575,678564,575,19270,12955,170488,1726,0,0,0,0,0,1726,0,0,0,12955,30497,2530094,0,581932
3,0,0,0,0,0,0,1236835,2370097,2167,983668,2707,11095,43666,42353,2167,0,0,0,0,0,2167,0,0,0,43666,53407,2370097,0,1236835
4,0,0,0,0,0,0,9795,0,82,359086,254,112519,6325,8995,82,0,0,0,0,0,82,0,0,0,6325,117460,746746,0,9795
5,0,0,0,0,0,0,8784,0,2759,539251,2759,781776,781776,28836,2848,0,0,0,0,0,2848,0,0,0,781776,781776,781776,0,8784
6,0,0,0,0,0,0,668133,668133,929,227722,929,144,13290,59292,3188,0,0,0,0,0,3188,0,0,0,13290,13290,668133,0,668133
7,0,0,0,0,0,0,25974,0,7039,366681,8005,336134,7375,7039,7039,0,0,0,0,0,7039,0,0,0,7375,336134,366681,0,25974
8,0,0,0,0,0,0,567915,0,207,250570,207,68,5534,16237,0,0,0,0,0,0,0,0,0,0,5534,5602,567915,0,567915


In [ ]:
'''Introducing air temperature rate of change at hourly and monthly intervals.'''

In [6]:
train.drop('hourly_roc', axis=1, inplace=True)

In [5]:
'''Creating a groupby object in order to compute the monthly rate of change in air_temperature. 
After consulting multiple kernels, it seems as though air_temperature is an important feature
to model.
'''

roc_month = train.groupby(['site_id', 'month'])['air_temperature'].agg(np.mean).reset_index('month').reset_index()

roc_month['dif']=0.00
for i in range(len(roc_month)-1):
    roc_month['dif'][i] = abs((roc_month['air_temperature'][i+1])-(roc_month['air_temperature'][i]))

In [6]:
temp = train.copy()

In [7]:
cols = ['site_id', 'month']
temp = temp.join(roc_month.set_index(cols), on=cols, lsuffix='temp_', rsuffix='roc_')

In [5]:
# def add_roc(x):
#     global roc_month
#     roc_df = roc_month.apply(lambda y: y['dif'] if
#                             (x['site_id']==y['site_id'] and x['month']==y['month'])
#                             else None, axis=1)
#     roc_df = roc_df.dropna(axis=0, how='all')
#     if roc_df.empty:
#         roc_df=np.nan
#     else:
#         roc_df = roc_df.to_string(index=False)
    
#     return roc_df

# train['month_roc'] = train.apply(add_roc, axis=1)

In [8]:
# Hourly air_temperature rate of change

roc = train.groupby(['site_id','hour'])['air_temperature'].agg(np.mean).to_frame().reset_index('hour').reset_index()

roc['dif']=0.00
for i in range(len(roc)-1):
    roc['dif'][i] = abs((roc['air_temperature'][i+1])-(roc['air_temperature'][i]))
    

In [9]:
temp2=train.copy()

In [10]:
cols = ['site_id', 'hour']
temp2 = temp2.join(roc.set_index(cols), on=cols, lsuffix='temp_', rsuffix='roc_')

In [11]:
train['month_roc'] = temp['dif']
train['hour_roc'] = temp2['dif']

##### Creating lag features using our 'timestamp' feature

In [7]:
train['timestamp'] = pd.to_datetime(train['timestamp'], errors='coerce')

In [8]:
'''The idea is to align the timestamp by peak air temperature, based on an assumption that the highest
air temperature should appear at around 14:00.'''

# calculate ranks of hourly temperatures within date/site_id chunks
train['temp_rank'] = train.groupby(['site_id', train.timestamp.dt.date])['air_temperature'].rank('average')

# create a dataframe of site_ids (0-16) x mean hour rank of temperature within day (0-23)
df_2d = train.groupby(['site_id', train.timestamp.dt.hour])['temp_rank'].mean().unstack(level=1)

# Subtract the columnID of temperature peak by 14, getting the timestamp alignment gap.
site_ids_offsets = pd.Series(df_2d.values.argmax(axis=1) - 14)
site_ids_offsets.index.name = 'site_id'

def timestamp_align(df):
    df['offset'] = df.site_id.map(site_ids_offsets)
    df['timestamp_aligned'] = (df.timestamp - pd.to_timedelta(df.offset, unit='H'))
    df['timestamp'] = df['timestamp_aligned']
    del df['timestamp_aligned']
    return df

In [9]:
'''function to add lag features''' #https://www.kaggle.com/nz0722/aligned-timestamp-lgbm-by-meter-type
def add_lag_feature(train, window=3):
    group_df = train.groupby('site_id')
    cols = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_direction', 'wind_speed']
    rolled = group_df[cols].rolling(window=window, min_periods=0)
    lag_mean = rolled.mean().reset_index().astype(np.float16)
    lag_max = rolled.max().reset_index().astype(np.float16)
    lag_min = rolled.min().reset_index().astype(np.float16)
    lag_std = rolled.std().reset_index().astype(np.float16)
    for col in cols:
        train[f'{col}_mean_lag{window}'] = lag_mean[col]
        train[f'{col}_max_lag{window}'] = lag_max[col]
        train[f'{col}_min_lag{window}'] = lag_min[col]
        train[f'{col}_std_lag{window}'] = lag_std[col]

In [10]:
train = timestamp_align(train)
# train = train.groupby('site_id').apply(lambda group: group.interpolate(limit_direction='both'))

add_lag_feature(train, window=3)
add_lag_feature(train, window=72)

In [11]:
train.head()

,building_id,meter,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,hour,day,weekday,month,day_of_year,wind_chill,heat_index,feels,radiation,altitude,solar_spec,random,age,timestamp,temp_rank,offset,air_temperature_mean_lag3,air_temperature_max_lag3,air_temperature_min_lag3,air_temperature_std_lag3,cloud_coverage_mean_lag3,cloud_coverage_max_lag3,cloud_coverage_min_lag3,cloud_coverage_std_lag3,dew_temperature_mean_lag3,dew_temperature_max_lag3,dew_temperature_min_lag3,dew_temperature_std_lag3,precip_depth_1_hr_mean_lag3,precip_depth_1_hr_max_lag3,precip_depth_1_hr_min_lag3,precip_depth_1_hr_std_lag3,sea_level_pressure_mean_lag3,sea_level_pressure_max_lag3,sea_level_pressure_min_lag3,sea_level_pressure_std_lag3,wind_direction_mean_lag3,wind_direction_max_lag3,wind_direction_min_lag3,wind_direction_std_lag3,wind_speed_mean_lag3,wind_speed_max_lag3,wind_speed_min_lag3,wind_speed_std_lag3,air_temperature_mean_lag72,air_temperature_max_lag72,air_temperature_min_lag72,air_temperature_std_lag72,cloud_coverage_mean_lag72,cloud_coverage_max_lag72,cloud_coverage_min_lag72,cloud_coverage_std_lag72,dew_temperature_mean_lag72,dew_temperature_max_lag72,dew_temperature_min_lag72,dew_temperature_std_lag72,precip_depth_1_hr_mean_lag72,precip_depth_1_hr_max_lag72,precip_depth_1_hr_min_lag72,precip_depth_1_hr_std_lag72,sea_level_pressure_mean_lag72,sea_level_pressure_max_lag72,sea_level_pressure_min_lag72,sea_level_pressure_std_lag72,wind_direction_mean_lag72,wind_direction_max_lag72,wind_direction_min_lag72,wind_direction_std_lag72,wind_speed_mean_lag72,wind_speed_max_lag72,wind_speed_min_lag72,wind_speed_std_lag72
0,0,Electricity,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,4,1,1,83.6,77.6,77.0,1414.0,-51.78,NaN,24802456,8.0,2015-12-31 18:00:00,12.5,6,25.0,25.0,25.0,NaN,6.0,6.0,6.0,NaN,20.0,20.0,20.0,NaN,NaN,NaN,NaN,NaN,1019.5,1019.5,1019.5,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN,25.0,25.0,25.0,NaN,6.0,6.0,6.0,NaN,20.0,20.0,20.0,NaN,NaN,NaN,NaN,NaN,1019.5,1019.5,1019.5,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,NaN
1,1,Electricity,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,4,1,1,83.6,77.6,77.0,1414.0,-51.78,NaN,61316517,12.0,2015-12-31 19:00:00,12.5,6,25.0,25.0,25.0,0.0,6.0,6.0,6.0,0.0,20.0,20.0,20.0,0.0,NaN,NaN,NaN,NaN,1019.5,1019.5,1019.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,6.0,6.0,6.0,0.0,20.0,20.0,20.0,0.0,NaN,NaN,NaN,NaN,1019.5,1019.5,1019.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Electricity,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,4,1,1,83.6,77.6,77.0,1414.0,-51.78,NaN,63831873,25.0,2015-12-31 20:00:00,12.5,6,25.0,25.0,25.0,0.0,6.0,6.0,6.0,0.0,20.0,20.0,20.0,0.0,NaN,NaN,NaN,NaN,1019.5,1019.5,1019.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,6.0,6.0,6.0,0.0,20.0,20.0,20.0,0.0,NaN,NaN,NaN,NaN,1019.5,1019.5,1019.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Electricity,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,4,1,1,83.6,77.6,77.0,1414.0,-51.78,NaN,56365151,14.0,2015-12-31 21:00:00,12.5,6,25.0,25.0,25.0,0.0,6.0,6.0,6.0,0.0,20.0,20.0,20.0,0.0,NaN,NaN,NaN,NaN,1019.5,1019.5,1019.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,6.0,6.0,6.0,0.0,20.0,20.0,20.0,0.0,NaN,NaN,NaN,NaN,1019.5,1019.5,1019.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Electricity,0.0,0,Education,-14465,1975.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0,0,1,4,1,1,83.6,77.6,77.0,1414.0,-51.78,NaN,4040317,41.0,2015-12-31 22:00:00,12.5,6,25.0,25.0,25.0,0.0,6.0,6.0,6.0,0.0,20.0,20.0,20.0,0.0,NaN,NaN,NaN,NaN,1019.5,1019.5,1019.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,6.0,6.0,6.0,0.0,20.0,20.0,20.0,0.0,NaN,NaN,NaN,NaN,1019.5,1019.5,1019.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
train.drop(['timestamp', 'temp_rank', 'offset',
            'precip_depth_1_hr_mean_lag3', 'precip_depth_1_hr','air_temperature_std_lag3',
            'cloud_coverage_mean_lag3', 'cloud_coverage_min_lag3',
            'dew_temperature_min_lag3', 'sea_level_pressure_min_lag3',
           'sea_level_pressure_std_lag3', 'sea_level_pressure_max_lag3',
           'precip_depth_1_hr_std_lag3','dew_temperature_std_lag3', 'dew_temperature_max_lag3',
           'cloud_coverage_max_lag3','precip_depth_1_hr_max_lag3', 'precip_depth_1_hr_min_lag3',
           'air_temperature_min_lag3','wind_direction_max_lag3','wind_direction_min_lag3','wind_direction_std_lag3',
           'wind_speed_max_lag3','wind_speed_min_lag3','wind_speed_std_lag3','cloud_coverage_max_lag72','cloud_coverage_min_lag72',
           'cloud_coverage_std_lag72', 'dew_temperature_max_lag72','dew_temperature_min_lag72','dew_temperature_std_lag72',
           'precip_depth_1_hr_max_lag72','precip_depth_1_hr_min_lag72','precip_depth_1_hr_std_lag72', 'sea_level_pressure_max_lag72',
           'sea_level_pressure_min_lag72', 'sea_level_pressure_std_lag72','wind_direction_max_lag72','wind_direction_min_lag72',
           'wind_direction_std_lag72','wind_speed_max_lag72','wind_speed_min_lag72','wind_speed_std_lag72'],axis=1,
           inplace=True)

In [13]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object and col_type.name != 'category' and 'datetime' not in col_type.name:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        elif 'datetime' not in col_type.name:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [14]:
train = reduce_mem_usage(train)

Memory usage of dataframe is 4819.89 MB
Memory usage after optimization is: 1831.56 MB
Decreased by 62.0%


In [15]:
train.to_csv('train_to_impute.csv')